In [147]:
import pandas as pd
import numpy as np
import quandl 
import datetime
import math
import warnings
warnings.filterwarnings("ignore")

In [319]:
#import excel file
df = pd.read_excel("Investable Universe v1.xlsx", header = 30)

#fixing UAL and GOOGL ticker
df["Exchange:Ticker"][2] = "NasdaqGS:GOOG.L"
df["Exchange:Ticker"][822] = "NasdaqGS:UAL"

In [320]:
#import price data
df1 = pd.read_csv("price data.csv")

#clean columns
df1 = df1[df1.columns[~df1.columns.str.contains('Unnamed:')]]
del df1["22/10/2018"]

In [321]:
#create log return columns and delete price columns

df2 = df1.copy()

for i in range(0,len(df2.columns[1:])):
    try:
        name = df2.columns[1:][i] + "_return"
        df2[name] = np.log(df2[df2.columns[1:][i]] / df2[df2.columns[1:][i+1]])
    except ValueError:
        continue
df2.drop(df2.columns[1:783], axis=1, inplace=True)

In [334]:
#Industries and geographies for correlations

primary_industry = ["Information Technology", "Consumer Discretionary", "Financials", "Energy",
                   "Healthcare", "Consumer Staples", "Telecommunication Services", "Industrials",
                   "Materials", "Utilities", "Real Estate", "NaN"]

market = ["North America", "Developed Asia ex-Japan", "Japan", "Developed Europe", "NaN"]

In [341]:
def correlation (df, df2, industry = "NaN", geography = "NaN"):
    if geography == "NaN":
        if industry == "NaN":
            company_list = df[df["Investable (1 = Yes, 0 = No)"] == 1]["Exchange:Ticker"]
            industry = "All Industries"
        else:
            company_list = df[df["Primary Industry"] == industry][df["Investable (1 = Yes, 0 = No)"] == 1]["Exchange:Ticker"]
        geography = "Global"
    
    elif industry == "NaN":
        company_list = df[df["Market"] == geography][df["Investable (1 = Yes, 0 = No)"] == 1]["Exchange:Ticker"]
        industry = "All Industries"
    
    else:
        company_list = df[df["Primary Industry"] == industry][df["Market"] == geography][df["Investable (1 = Yes, 0 = No)"] == 1]["Exchange:Ticker"]
    
    
    
    company_list = company_list.values.T.tolist()
    
    df_industry = df2.loc[df2['Ticker'].isin(company_list)]
    df_industry = df_industry.set_index("Ticker")
    df_industry = df_industry.T
    df_industry = df_industry.corr()
    
    mask = df_industry != 1
    df0 = df_industry[mask]
    df0 = df0.stack()
    mean = df0.mean()
    correlation_dict = {"Geography" : geography, "Industry" : industry, "Dispersion" : 1-mean}
    return correlation_dict

In [342]:
def correlation_table(primary_industry, market):
    b = []
    
    for i in market:
        for x in (primary_industry):
            a = correlation(df, df2, x, i)
            b.append(a)
    
    c = pd.DataFrame(b)
    c = c[["Geography", "Industry", "Dispersion"]]
    return c

In [343]:
df_table = correlation_table(primary_industry, market)

In [344]:
df_table

,Geography,Industry,Dispersion
0,North America,Information Technology,0.623860
1,North America,Consumer Discretionary,0.702839
2,North America,Financials,0.454369
3,North America,Energy,0.377681
4,North America,Healthcare,0.693442
5,North America,Consumer Staples,0.197871
6,North America,Telecommunication Services,0.261233
7,North America,Industrials,0.580976
8,North America,Materials,0.562015
9,North America,Utilities,0.137332


In [346]:
df_table.to_excel("Dispersion.xlsx")